In [ ]:
import json
import re
from pathlib import Path
from typing import List, Dict, Any, Tuple, Optional

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


MODEL_ID = "microsoft/phi-2"
SYSTEM_PROMPT = "You are a careful academic assistant. Be precise and return strict JSON."


def init_pipeline() -> Tuple[Any, Any, int]:
        using_gpu = torch.cuda.is_available()
        device = 0 if using_gpu else -1
        print(f"Using device: {'GPU' if device == 0 else 'CPU'}")

        tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
        if tokenizer.pad_token_id is None:
                tokenizer.pad_token_id = tokenizer.eos_token_id

        model = AutoModelForCausalLM.from_pretrained(
                MODEL_ID,
                device_map="auto" if using_gpu else None,
                torch_dtype=torch.float16 if using_gpu else torch.float32,
                low_cpu_mem_usage=True,
                trust_remote_code=True,
        )

        text_gen = pipeline(
                "text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=768,
                do_sample=True,
                temperature=0.7,
                top_p=0.95,
                top_k=50,
        )

        return tokenizer, text_gen, tokenizer.eos_token_id


def render_chat(system: str, user: str, tokenizer) -> str:
        messages = [
                {"role": "system", "content": system},
                {"role": "user", "content": user},
        ]
        apply_chat = getattr(tokenizer, "apply_chat_template", None)
        if callable(apply_chat):
                try:
                        return apply_chat(messages, tokenize=False)
                except Exception:
                        pass
        return f"System: {system}\nUser: {user}\nAssistant:"


def build_detection_prompt(submission: str, few_shots: list, tokenizer) -> str:
        shot_texts = []
        for s in few_shots:
                shot_texts.append(
                        f'Submission: """{s.get("final_submission","")}"""\n'
                        f'Your analysis (2–4 bullet points): <analysis>\n'
                        f'Label: {s.get("label_type","")}\n'
                )
        examples_block = "\n\n".join(shot_texts) if shot_texts else "/* no examples available */"
        user_block = f"""You are an AI text-source classifier for academic integrity.
Decide whether the student submission is Human, AI, or Hybrid (AI-assisted).
Guidelines:
- Consider discourse features (specificity, subjectivity, personal context), style consistency, local/global coherence, repetitiveness, and cliché patterns.
- Hybrid = meaningful human writing with some AI assistance, or explicit admission of mixed use.
Examples:
{examples_block}
Now analyze the NEW submission and respond in plain text with the following structure:
Label: ...
Rationale:
- point 1
- point 2
Flags: ...
NEW submission:
\"\"\"{submission}\"\"\""""
        return render_chat(SYSTEM_PROMPT, user_block, tokenizer)


def build_feedback_prompt(domain: str, assignment_prompt: str, rubric_text: str, submission: str, tokenizer) -> str:
        user_block = f"""You are a supportive assessor. Provide actionable feedback aligned to the rubric.
Return plain structured text only (no JSON, no files).
Sections to include:
1) Overall Summary: 2–4 sentences on strengths and priorities.
2) Criteria Feedback: for each rubric criterion include:
   - Criterion
   - Rating (excellent, good, average, needs_improvement, poor)
   - Evidence (1–3 bullet points citing excerpts or behaviors)
   - Improvement Tip: one concrete step
3) Suggested Grade: short string (optional)
Context:
- Domain: {domain}
- Assignment prompt: {assignment_prompt}
Rubric (verbatim):
{rubric_text}
Student submission:
\"\"\"{submission}\"\"\""""
        return render_chat(SYSTEM_PROMPT, user_block, tokenizer)


def format_rubric(rubric: Dict[str, Any]) -> str:
        formatted_rubric = f"Rubric ID: {rubric['rubric_id']}\n\nCriteria:\n"
        for item in rubric['criteria']:
                formatted_rubric += f"Criterion: {item['criterion_id']}\nName: {item['name']}\nDescription: {item['description']}\nPerformance Descriptors:\n"
                for key, val in item['performance_descriptors'].items():
                        formatted_rubric += f"  - {key}: {val}\n"
        return formatted_rubric


def generate_text(text_gen, prompt_text: str, eos_token_id: int) -> str:
        outputs = text_gen(prompt_text, return_full_text=False, eos_token_id=eos_token_id)
        return outputs[0]["generated_text"].strip()


def _normalize_label(label: str) -> Optional[str]:
        if not label:
                return None
        l = label.strip().lower()
        if "human" in l:
                return "Human"
        if l == "ai" or " ai" in l or l.startswith("ai"):
                return "AI"
        if "hybrid" in l:
                return "Hybrid"
        return None


def extract_detection_fields(text: str) -> Dict[str, Any]:
        # Try JSON first
        try:
                obj = json.loads(text)
                label = _normalize_label(obj.get("label"))
                rationale = obj.get("rationale")
                if isinstance(rationale, str):
                        rationale = [r.strip("- ").strip() for r in rationale.split("\n") if r.strip()]
                flags = obj.get("flags") if isinstance(obj.get("flags"), list) else None
                return {
                        "label": label,
                        "rationale": rationale if isinstance(rationale, list) else None,
                        "flags": flags,
                }
        except Exception:
                pass

        # Fallback: regex extraction from plain text
        label_match = re.search(r"(?im)^\s*Label\s*:\s*([^\n\r]+)", text)
        label = _normalize_label(label_match.group(1) if label_match else None)

        # Rationale block
        rat_block = None
        rat_start = re.search(r"(?im)^\s*Rationale\s*:", text)
        if rat_start:
                start_idx = rat_start.end()
                end_match = re.search(r"(?im)^\s*Flags\s*:", text[start_idx:])
                rat_block = text[start_idx:start_idx + end_match.start()] if end_match else text[start_idx:]

        rationale = None
        if rat_block:
                lines = [ln.strip() for ln in rat_block.splitlines()]
                rationale = [re.sub(r"^\s*[-•]\s*", "", ln).strip() for ln in lines if ln.strip()]

        # Flags line
        flags = None
        flags_match = re.search(r"(?im)^\s*Flags\s*:\s*([^\n\r]+)", text)
        if flags_match:
                flags = [f.strip() for f in re.split(r"[;,]", flags_match.group(1)) if f.strip()]

        return {"label": label, "rationale": rationale, "flags": flags}


def attempt_json(text: str) -> Optional[Dict[str, Any]]:
        try:
                return json.loads(text)
        except Exception:
                return None


def run_and_export_predictions(
        tokenizer,
        text_gen,
        eos_token_id: int,
        data_paths: List[Path],
        output_dir: Path = Path("outputs")
) -> Tuple[Path, Path]:
        output_dir.mkdir(parents=True, exist_ok=True)
        detect_path = output_dir / "predictions_detection.jsonl"
        feedback_path = output_dir / "predictions_feedback.jsonl"

        all_detection_data = []
        all_feedback_data = []

        for data_path in data_paths:
                if not data_path.exists():
                        print(f"Warning: {data_path} not found; skipping.")
                        continue

                with data_path.open() as f:
                        data = json.load(f)

                rubric_text = format_rubric(data["rubric"])
                few_shots = data.get("few_shots", [])
                domain = data.get("domain", "General")
                assign_prompt = data.get("prompt", "Analyze student submission and provide detailed feedback")

                for i, submission in enumerate(data.get("submissions", []), 1):
                        submission_text = submission["final_submission"]
                        label_true = submission.get("label_type", "Unknown")

                        # Feedback
                        feedback_prompt = build_feedback_prompt(
                                domain=domain,
                                assignment_prompt=assign_prompt,
                                rubric_text=rubric_text,
                                submission=submission_text,
                                tokenizer=tokenizer
                        )
                        feedback_text = generate_text(text_gen, feedback_prompt, eos_token_id)
                        feedback_obj = attempt_json(feedback_text)

                        all_feedback_data.append({
                                "dataset": data_path.name,
                                "submission_index": i,
                                "feedback_raw": feedback_text,
                                "feedback_json": feedback_obj
                        })

                        # Detection
                        detection_prompt = build_detection_prompt(submission_text, few_shots, tokenizer)
                        detection_text = generate_text(text_gen, detection_prompt, eos_token_id)
                        det = extract_detection_fields(detection_text)

                        all_detection_data.append({
                                "dataset": data_path.name,
                                "submission_index": i,
                                "label_true": label_true,
                                "label_pred": det.get("label"),
                                "rationale": det.get("rationale"),
                                "flags": det.get("flags"),
                                "raw": detection_text
                        })

        with detect_path.open("w", encoding="utf-8") as df:
            json.dump(all_detection_data, df, indent=4, ensure_ascii=False)

        with feedback_path.open("w", encoding="utf-8") as ff:
             json.dump(all_feedback_data, ff, indent=4, ensure_ascii=False)


        print(f"Wrote: {detect_path}")
        print(f"Wrote: {feedback_path}")
        return detect_path, feedback_path


def main() -> None:
        tokenizer, text_gen, eos_token_id = init_pipeline()
        data_paths = [Path("accounting.json"), Path("engineering.json"),Path("it.json"),Path("psychology.json"),Path("teaching.json")]
        run_and_export_predictions(tokenizer, text_gen, eos_token_id, data_paths)


if __name__ == "__main__":
        main()

Using device: GPU


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Wrote: outputs/predictions_detection.jsonl
Wrote: outputs/predictions_feedback.jsonl
